In [ ]:
# Install RDKit
!pip install rdkit

In [ ]:
# Import Libraries
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import random

print(f"RDKit version: {rdkit.__version__}")

In [ ]:
#Load and Prepare Data
# Load dataset
url = 'https://raw.githubusercontent.com/dataprofessor/data/master/delaney.csv'
sol_df = pd.read_csv(url)

# Display first few rows of the dataset
print("Dataset head:")
display(sol_df.head())

# Convert SMILES to RDKit molecules
mol_data = [Chem.MolFromSmiles(element) for element in sol_df.SMILES]
print(f"\nTotal molecules processed: {len(mol_data)}")
print(f"First molecule: {mol_data[0]}")

In [ ]:
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit import Chem

def draw_molecules_grid(smiles_dict, n_cols=3, img_size=(300, 300)):
    """Draw molecules in a grid layout with labels"""
    mols = []
    legends = []

    # Convert SMILES to RDKit molecules
    for name, smiles in smiles_dict.items():
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            # Generate 2D coordinates for the molecule
            AllChem.Compute2DCoords(mol)
            mols.append(mol)
            legends.append(name)

    # Calculate number of rows needed
    n_rows = (len(mols) + n_cols - 1) // n_cols

    # Create the grid image
    img = Draw.MolsToGridImage(mols,
                              molsPerRow=n_cols,
                              subImgSize=img_size,
                              legends=legends,
                              returnPNG=False)

    # Display using matplotlib
    plt.figure(figsize=(15, 5*n_rows))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
sample_molecules = dict(zip(
    [f'Molecule_{i+1}' for i in range(5)],  # Names
    sol_df['SMILES'].head(5)  # SMILES strings
))

# Visualize these molecules
draw_molecules_grid(sample_molecules)

In [ ]:
#Calculate Molecular Descriptors
mol_descriptors_np = np.arange(1,1)

for mol in mol_data:
    clogP = Descriptors.MolLogP(mol)
    mol_weight = Descriptors.MolWt(mol)
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)

    row = np.array([clogP, mol_weight, num_rotatable_bonds])

    if mol_descriptors_np.size == 0:
        mol_descriptors_np = row
    else:
        mol_descriptors_np = np.vstack([mol_descriptors_np, row])

print("Molecular descriptors shape:", mol_descriptors_np.shape)

Molecular descriptors shape: (1144, 3)


In [ ]:
#Calculate Aromatic Proportion
aromatic_arr = []

for mol in mol_data:
    total_atoms = mol.GetNumAtoms()
    aromatic_count = 0
    for i in range(total_atoms):
        if mol.GetAtomWithIdx(i).GetIsAromatic():
            aromatic_count += 1

    aromatic_arr.append(aromatic_count/Descriptors.HeavyAtomCount(mol))

aromatic_np = np.array(aromatic_arr).reshape(-1,1)
print("Aromatic proportions shape:", aromatic_np.shape)

Aromatic proportions shape: (1144, 1)


In [ ]:
#Create Final Dataset
# Combine all descriptors
all_descriptors_np = np.hstack([mol_descriptors_np, aromatic_np])

# Create descriptors dataframe
descriptors_df = pd.DataFrame(all_descriptors_np,
                            columns=['cLogP', 'MolWeight', 'numRotatableBonds', 'AromaticProportion'])

# Add solubility values
descriptors_df = pd.concat([descriptors_df, sol_df["measured log(solubility:mol/L)"]], axis=1)
descriptors_df.rename(columns={'measured log(solubility:mol/L)': 'log(solubility:mol/L)'}, inplace=True)

# Display dataset info
print("Final dataset info:")
display(descriptors_df.describe())

In [ ]:
#Data Preparation for Modeling
# Set seed for reproducibility
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

# Prepare features and target
X = descriptors_df[['cLogP', 'MolWeight', 'numRotatableBonds', 'AromaticProportion']].values
Y = descriptors_df[['log(solubility:mol/L)']].values

# Normalize data
scaler = MinMaxScaler()
normalized_X = scaler.fit_transform(X)
normalized_Y = scaler.fit_transform(Y).ravel()

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(normalized_X, normalized_Y,
                                                    test_size=0.2, random_state=SEED)

print("Data shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"Y_train: {Y_train.shape}")
print(f"Y_test: {Y_test.shape}")

In [ ]:
#Linear Regression Model
# Train and evaluate linear regression model
lr_model = linear_model.LinearRegression()
lr_model.fit(X_train, Y_train)

Y_pred_test_lr = lr_model.predict(X_test)
lin_reg_mse = mean_squared_error(Y_test, Y_pred_test_lr)
lin_reg_r2 = r2_score(Y_test, Y_pred_test_lr)

print(f'Linear regression - MSE: {lin_reg_mse:.4f}')
print(f'Linear regression - R²: {lin_reg_r2:.4f}')

# Plot results
plt.figure(figsize=(10, 6))
slope_lin, intercept_lin = np.polyfit(Y_test, Y_pred_test_lr, 1)
plt.scatter(x=Y_test, y=Y_pred_test_lr, color='blue', s=25, alpha=0.5)
plt.plot(Y_test, slope_lin*Y_test + intercept_lin, color='red', alpha=0.6)
plt.xlabel('Experimental Solubility', fontsize=12)
plt.ylabel('Predicted Solubility', fontsize=12)
plt.title('Multiple Linear Regression', fontsize=15)
plt.show()

In [ ]:
#TODO: Random Forest Model
# Train and evaluate random forest model


In [ ]:
# Model Comparison Summary

Reference: ESOL:  Estimating Aqueous Solubility Directly from Molecular Structure (https://pubs.acs.org/doi/10.1021/ci034243x)


In [ ]:
# TODO: Predict solubility of mefenamic acid in water